In [69]:
!pip install mlflow pyvi minio -q
import mlflow

In [70]:
mlflow.set_tracking_uri("http://36.50.135.226:7893/")

# from mlflow.tracking import MlflowClient

# client = MlflowClient()
# client.restore_experiment(experiment_id="2")

mlflow.set_experiment(experiment_id="6")

<Experiment: artifact_location='s3://mlflow/5', creation_time=1744189879834, experiment_id='6', last_update_time=1744189879834, lifecycle_stage='active', name='RegisterModel', tags={}>

In [71]:
import os

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://36.50.135.226:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"

# MinIO

In [72]:
from minio import Minio
from minio.error import S3Error
import glob

In [73]:
# def upload_to_minio(local_folder, minio_folder):
#     MINIO_ENDPOINT = "http://160.191.244.13:9000"
#     ACCESS_KEY = "minio"
#     SECRET_KEY = "minio123"
#     BUCKET_NAME = "mlflow"
#     MINIO_FOLDER = "model/ViT_GPT2_Vi"


#     # Kết nối MinIO
#     minio_client = Minio(
#         MINIO_ENDPOINT.replace("http://", "").replace("https://", ""),
#         access_key=ACCESS_KEY,
#         secret_key=SECRET_KEY,
#         secure=False,
#     )

#     # Kiểm tra và tạo bucket nếu chưa tồn tại
#     found = minio_client.bucket_exists(BUCKET_NAME)
#     if not found:
#         minio_client.make_bucket(BUCKET_NAME)
#         print(f"Bucket '{BUCKET_NAME}' được tạo thành công.")

#     # Tìm tất cả file trong thư mục local
#     files = glob.glob(f"{local_folder}/*")
#     for file_path in files:
#         file_name = os.path.basename(file_path)
#         minio_path = f"{minio_folder}/{file_name}"

#         try:
#             minio_client.fput_object(BUCKET_NAME, minio_path, file_path)
#             print(f"✅ Đã upload: {file_name} → {minio_path}")
#         except S3Error as e:
#             print(f"❌ Lỗi khi upload {file_name}: {e}")


# Training

In [74]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import transformers
from transformers import VisionEncoderDecoderModel, ViTImageProcessor
import torch
from PIL import Image
import torch.nn as nn
import cv2
import torchvision
from tqdm.notebook import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification, ViTImageProcessor
from collections import OrderedDict
from transformers import GPT2Config, GPT2LMHeadModel
import mlflow
from mlflow.models import infer_signature
import mlflow.pytorch
import re
from pyvi import ViTokenizer
from torch.optim import AdamW

In [75]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [76]:
with open("requirements.txt") as f:
    pip_reqs = [line.strip() for line in f if line.strip()]

In [77]:
"""
1. Load data
"""
# 1. Load dữ liệu
bucket = 0
bucket_path = f"/kaggle/input/traffic-image-caption-split/output/bucket_{bucket}"

def load_data(bucket_path):  
    train_df = pd.read_json(os.path.join(bucket_path, 'train.json'))
    val_df = pd.read_json(os.path.join(bucket_path, 'val.json'))
    test_df = pd.read_json(os.path.join(bucket_path, 'test.json'))
    return train_df, val_df, test_df

train_df, val_df, test_df = load_data(bucket_path)
# 2. In kích thước từng file
print("Train size:", train_df.shape)
print("Validation size:", val_df.shape)
print("Test size:", test_df.shape)

# 3. Gộp lại thành một dataframe
df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# 4. Chọn các cột cần thiết và đổi tên
def rename_columns(df):
    return df[['original_url', 'search_query', 'short_caption']].rename(columns={
        'original_url': 'url',
        'search_query': 'search_query',
        'short_caption': 'caption'
    })
df_new = rename_columns(df)
train_df = rename_columns(train_df)
val_df = rename_columns(val_df)
test_df = rename_columns(test_df)

# 5. In ra 5 dòng đầu tiên
print(df_new.head())

Train size: (9838, 6)
Validation size: (1252, 6)
Test size: (1262, 6)
                                                 url  \
0  https://media.vietnamplus.vn/images/dadb342ab8...   
1  https://cdn.tuoitre.vn/zoom/700_700/4715847528...   
2  https://media1.nguoiduatin.vn/media/pham-trong...   
3  https://laodongthudo.vn/stores/news_dataimages...   
4  https://binhphuoc.gov.vn/uploads/binhphuoc/stp...   

                          search_query  \
0       nhường đường cho xe cứu thương   
1  lối sang đường dành cho người đi bộ   
2         nắng nóng đường phố việt nam   
3            trật tự giao thông ngã tư   
4               xe máy đậu trên vỉa hè   

                                             caption  
0  Giao thông tĩnh lặng, nhiều xe cứu thương đậu ...  
1  Giao thông vắng vẻ, có người băng qua đường tạ...  
2  Giao thông đường phố khá vắng vẻ.  Biển báo và...  
3  Giao thông đông xe máy. Biển báo giới hạn chiề...  
4  Tình trạng giao thông khá vắng vẻ. Biển báo ở ...  


In [78]:
"""
3. Load feature extractor
"""
feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

def load_image(image_url):
    try:
        response = requests.get(image_url, timeout=10)
        if response.status_code != 200:
            print(f"Error: Failed to fetch image, status code {response.status_code}")
            return None

        image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

        if image is None:
            print("Error: OpenCV could not decode the image.")
            return None

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image_rgb
    except requests.exceptions.RequestException as e:
        print(f"Error: Request failed - {e}")
        return None

In [79]:
"""
4. Load Tokenizer
"""
from transformers import AutoTokenizer

# Load tokenizer của BartPho
# tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")

# Kiểm tra vocab size
vocab_size = tokenizer.vocab_size
print(f"VOCAB SIZE: {vocab_size}")

VOCAB SIZE: 64000


In [80]:
# In ra các special tokens
print("Special Tokens:", tokenizer.special_tokens_map)

Special Tokens: {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}


In [81]:
# # Thêm token mới nếu chưa có
# if "<|startoftext|>" not in tokenizer.get_vocab():
#     tokenizer.add_special_tokens({'bos_token': '<|startoftext|>'})

# vocab_size = tokenizer.vocab_size
# print(vocab_size)

In [82]:
"""
5. Các hàm tiền xử lý caption
"""
def clean_text(text: str) -> str:
    return re.sub(r"[^\w\s,!?.]", "", text).strip()

def to_lowercase(text: str) -> str:
    return text.lower()

def join_vietnamese_compounds(text: str) -> str:
    return ViTokenizer.tokenize(text)

def caption_preprocess(text: str) -> str:
    text = clean_text(text)
    text = to_lowercase(text)
    text = join_vietnamese_compounds(text)
    return text

In [83]:
text = caption_preprocess("Kiểm tra phân tách từ")

# Tách token
tokens = tokenizer.tokenize(text)

# Chuyển token thành ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)

text = tokenizer.decode(token_ids, skip_special_tokens=True)
# In kết quả
print("List Word (Tokenized):", tokens)
print("List Token ID:", token_ids)
print("Text:", text)

List Word (Tokenized): ['kiểm_tra', 'phân_tách', 'từ']
List Token ID: [342, 31166, 39]
Text: kiểm_tra phân_tách từ


In [84]:
"""
6. Tiền xử lý input model
"""
def process_data(image_url, caption):
    try:
        img_array = load_image(image_url)
        if img_array is None:
            return None
        
        pixel_values = feature_extractor(img_array, return_tensors="pt")["pixel_values"].squeeze(0)
        caption = caption_preprocess(caption)
        tokenized_caption = tokenizer(caption, padding="max_length", max_length=315, truncation=True)
        
        return {
            "pixel_values": pixel_values,
            "input_ids": torch.tensor(tokenized_caption["input_ids"]),
            "attention_mask": torch.tensor(tokenized_caption["attention_mask"])
        }
    except Exception as e:
        print(f"Error processing data: {e}")
        return None

In [85]:
"""
7. Tạo tập dữ liệu huấn luyện
"""
class ImageCaptionDataset(Dataset):
    def __init__(self, image_paths, captions):
        self.image_paths = image_paths
        self.captions = captions

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        data = process_data(self.image_paths[idx], self.captions[idx])
        if data is None:
            return self.__getitem__((idx + 1) % len(self))
        return data

def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    return {
        "pixel_values": torch.stack([item["pixel_values"] for item in batch]),
        "input_ids": torch.stack([item["input_ids"] for item in batch]),
        "attention_mask": torch.stack([item["attention_mask"] for item in batch])
    }

train_dataset = ImageCaptionDataset(train_df["url"], train_df["caption"])
val_dataset = ImageCaptionDataset(val_df["url"], val_df["caption"])

In [86]:
"""
8. Cấu hình LoRA
"""
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel

# Cấu hình LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],  # Các lớp trong GPT2
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [87]:
"""
9. Định nghĩa mô hình
"""
vit_model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
del vit_model.classifier
vit_model.vit.pooler = torch.nn.Sequential(OrderedDict([
    ('dense', torch.nn.Linear(in_features=768, out_features=768, bias=True)),
    ('activation', torch.nn.Tanh())
]))

config = GPT2Config.from_pretrained("gpt2")
config.add_cross_attention = True
config.vocab_size = vocab_size

gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2", config=config, ignore_mismatched_sizes=True)
gpt2_model.resize_token_embeddings(config.vocab_size)

# Áp dụng LoRA vào GPT2
gpt2_model = get_peft_model(gpt2_model, lora_config)
gpt2_model.print_trainable_parameters()  # kiểm tra số lượng tham số cần huấn luyện

model = VisionEncoderDecoderModel(encoder=vit_model.vit, decoder=gpt2_model)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['transformer.h.0.crossattention.c_attn.bias', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.0.crossattention.q_attn.bias', 'transformer.h.0.crossattention.q_attn.weight', 'transformer.h.0.ln_cross_attn.bias', 'transformer.h.0.ln_cross_attn.weight', 'transformer.h.1.crossattention.c_attn.bias', 'transformer.h.1.crossattention.c_attn.weight', 'transformer.h.1.crossattention.c_proj.bias', 'transformer.h.1.crossattention.c_proj.weight', 'transformer.h.1.crossattention.q_attn.bias', 'transformer.h.1.crossattention.q_attn.weight', 'transformer.h.1.ln_cross_attn.bias', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.10.crossattention.c_attn.bias', 'transformer.h.10.crossattention.c_attn.weight', 'transformer.h.10.crossattention.c_proj.bias', 'transformer.h.10.cros

trainable params: 1,179,648 || all params: 164,540,928 || trainable%: 0.7169


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [88]:
class LoRACaptionWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        super().__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def load_context(self, context):
        try:
            # Đường dẫn đầy đủ đến các thư mục con trong artifact
            model_path = context.artifacts["model_dir"]
            tokenizer_path = os.path.join(model_path, "tokenizer")
            feature_extractor_path = os.path.join(model_path, "feature_extractor")

            
            # Load các thành phần với đường dẫn chính xác
            self.model = VisionEncoderDecoderModel.from_pretrained(model_path).to(self.device)
            self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
            self.feature_extractor = ViTImageProcessor.from_pretrained(feature_extractor_path)
            
            # Đặt model vào chế độ eval
            self.model.eval()
        except Exception as e:
            raise ValueError(f"Error loading model: {str(e)}")

    def predict(self, context, model_input):
        try:
            # Xử lý đầu vào
            if isinstance(model_input, dict):
                image_url = model_input["url"][0] if "url" in model_input else model_input["image_path"][0]
            else:
                image_url = model_input.iloc[0]["url"] if "url" in model_input.columns else model_input.iloc[0]["image_path"]
            
            image = self.load_image(image_url)
            if image is None:
                return [""]
                
            # Tiền xử lý ảnh và tạo caption
            pixel_values = self.feature_extractor(images=image, return_tensors="pt").pixel_values.to(self.device)
            
            output_ids = self.model.generate(
                pixel_values,
                max_length=45,
                min_length=20,
                num_beams=4,
                do_sample=True,
                temperature=0.8,
                top_k=20,
                top_p=0.9,
                no_repeat_ngram_size=3,
                repetition_penalty=2.0,
                early_stopping=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                decoder_start_token_id=self.tokenizer.bos_token_id
            )
            
            caption = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
            return [caption.replace("_", " ").strip()]
            
        except Exception as e:
            print(f"[ERROR] during prediction: {e}")
            return [""]
   
    def load_image(self, image_url):
        try:
            import requests
            import numpy as np
            from PIL import Image
            from io import BytesIO
            
            response = requests.get(image_url, timeout=10)
            if response.status_code != 200:
                return None
                
            image = Image.open(BytesIO(response.content))
            return image.convert("RGB") if image else None
            
        except Exception as e:
            print(f"[ERROR] loading image: {e}")
            return None

/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [89]:
def log_datasets_to_mlflow():
    # Log vào MLflow dưới dạng bảng (MLflow 2.4+)
    mlflow.log_table(train_df, "datasets/train.json")
    mlflow.log_table(val_df, "datasets/val.json")
    mlflow.log_table(test_df, "datasets/test.json")

In [90]:
"""
10. Huấn luyện mô hình
"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def evaluate_model(model, val_dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            encoder_outputs = model.encoder(pixel_values=pixel_values)
            outputs = model.decoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                encoder_hidden_states=encoder_outputs.last_hidden_state,
                labels=input_ids
            )
            loss = outputs.loss
            total_loss += loss.item()
    return total_loss / len(val_dataloader)

def train_model(model, train_dataloader, val_dataloader, tokenizer, feature_extractor, lr=5e-5, epochs=1, epoch_count=1, model_name='BartPho_ViT_GPT2_LoRA_ICG', df_log=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Optimizer chỉ update các tham số có requires_grad=True (LoRA adapter)
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    # Thống kê tham số trainable
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())

    with mlflow.start_run(run_name=model_name):
        if df_log is not None:
            dataset = mlflow.data.from_pandas(df_log, targets="caption")
            mlflow.log_input(dataset, context="training")

        # Logging các siêu tham số
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("epoch_count", epoch_count)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("train_data_size", len(train_dataloader.dataset))
        mlflow.log_param("val_data_size", len(val_dataloader.dataset))
        mlflow.log_param("trainable_params", trainable_params)
        mlflow.log_param("total_params", total_params)
        mlflow.log_param("trainable_ratio", round(trainable_params / total_params, 4))
        mlflow.log_param("model_name", model_name)

        model.train()
        for epoch in range(epochs):
            total_loss = 0
            for batch in train_dataloader:
                pixel_values = batch["pixel_values"].to(device)
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)

                encoder_outputs = model.encoder(pixel_values=pixel_values)
                outputs = model.decoder(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    encoder_hidden_states=encoder_outputs.last_hidden_state,
                    labels=input_ids
                )
                loss = outputs.loss
                total_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            avg_loss = total_loss / len(train_dataloader)
            val_loss = evaluate_model(model, val_dataloader, device)

            # Log loss theo epoch
            mlflow.log_metric("train_loss", avg_loss, step=epoch)
            mlflow.log_metric("val_loss", val_loss, step=epoch)
            print(f"[Epoch {epoch+1}/{epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f}")

            # Lưu mô hình sau mỗi epoch
            epoch_save_path = f"/kaggle/working/{model_name}_epoch_{epoch+1}"
            model.save_pretrained(epoch_save_path)
            tokenizer.save_pretrained(os.path.join(epoch_save_path, "tokenizer"))
            feature_extractor.save_pretrained(os.path.join(epoch_save_path, "feature_extractor"))
            
            # Log toàn bộ thư mục như một artifact
            mlflow.log_artifacts(epoch_save_path, artifact_path=f"epoch_{epoch+1}")

        # Lưu model cuối cùng riêng biệt
        final_model_path = f"/kaggle/working/{model_name}_final"
        model.save_pretrained(final_model_path)
        tokenizer.save_pretrained(os.path.join(final_model_path, "tokenizer"))
        feature_extractor.save_pretrained(os.path.join(final_model_path, "feature_extractor"))
        mlflow.log_artifacts(final_model_path, artifact_path="final_model")

        # Log model dưới dạng pyfunc (nếu cần)
        # Đảm bảo bạn đã định nghĩa class ImageCaptionModel trước

        # Infer signature
        input_example = pd.DataFrame({"url": ["http://example.com/test.jpg"]})
        output_example = pd.DataFrame({"caption": ["Mô tả ảnh"]})
        
        # 2. Tạo model signature
        signature = infer_signature(
            input_example,
            output_example,
            params={"input_types": "string", "output_types": "string"}  # Chỉ định kiểu dữ liệu
        )
        mlflow.pyfunc.log_model(
            artifact_path="model",
            python_model=LoRACaptionWrapper(),
            artifacts={"model_dir": final_model_path},  # Đường dẫn đến thư mục chứa model, tokenizer và feature_extractor
            registered_model_name=model_name,
            signature=signature,
            # input_example=input_example,
            pip_requirements=pip_reqs
        )

In [103]:
# Tạo k Subset cho train, val
train_subset = Subset(train_dataset, range(train_df.shape[0]))  # train_df.shape[0]
val_subset = Subset(val_dataset, range(val_df.shape[0]))

# Tạo DataLoader cho các subset
train_dataloader = DataLoader(train_subset, batch_size=4, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader = DataLoader(val_subset, batch_size=4, shuffle=False, collate_fn=custom_collate_fn)

In [104]:
# """
# Load model cũ
# """
# # Đường dẫn thư mục chứa các tệp mô hình
# model_path = '/kaggle/input/bartpho_vit_gpt2_vi/pytorch/default/5'

# # Tải mô hình VisionEncoderDecoder
# model = VisionEncoderDecoderModel.from_pretrained(model_path, ignore_mismatched_sizes=True)

# model = model.to(device)

In [108]:
# Huấn luyện mô hình
train_model(model, train_dataloader, val_dataloader, tokenizer, feature_extractor, lr=5e-5, epochs=1, epoch_count=1, model_name='BartPho_ViT_GPT2_LoRA_ICG', df_log=df_new)

Error: OpenCV could not decode the image.
[Epoch 1/1] Train Loss: 15.1145 | Val Loss: 13.9766


/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/__init__.py:3212: UserWarning: An input example was not provided when logging the model. To ensure the model signature functions correctly, specify the `input_example` parameter. See https://mlflow.org/docs/latest/model/signatures.html#model-input-example for more details about the benefits of using input_example.
  color_warning(


Successfully registered model 'BartPho_ViT_GPT2_LoRA_ICG'.
2025/04/17 09:18:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BartPho_ViT_GPT2_LoRA_ICG, version 1
Created version '1' of model 'BartPho_ViT_GPT2_LoRA_ICG'.


🏃 View run BartPho_ViT_GPT2_LoRA_ICG at: http://36.50.135.226:7893/#/experiments/6/runs/c2da55b1378c42c0b95470c1c4ac3aab
🧪 View experiment at: http://36.50.135.226:7893/#/experiments/6


## Load model

In [109]:
# import mlflow
# import requests
# from PIL import Image
# from io import BytesIO

# # 1. Tải model từ MLflow
# model_uri = "models:/BartPho_ViT_GPT2_LoRA_ICG/1"
# model = mlflow.pyfunc.load_model(model_uri)
# # print(model.metadata)

# # 2. Chuẩn bị input theo đúng signature
# image_url = "https://hunghoangphat.vn/upload/images/NHÀ%20THÉP%20TIỀN%20CHẾ/nha-thep-tien-che-bai-do-xe%20(3).png"

# # 3. Gọi prediction ĐÚNG CÁCH
# result = model.predict({"url": image_url})  # Phải truyền dictionary với key 'url'

# # 4. Xử lý kết quả (output là dictionary với key 'caption')
# print("Generated caption:", result)

Some weights of the model checkpoint at /tmp/tmpserasi80/artifacts/BartPho_ViT_GPT2_LoRA_ICG_final were not used when initializing VisionEncoderDecoderModel: ['decoder.base_model.model.transformer.h.0.attn.c_attn.base_layer.bias', 'decoder.base_model.model.transformer.h.0.attn.c_attn.base_layer.weight', 'decoder.base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight', 'decoder.base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight', 'decoder.base_model.model.transformer.h.0.attn.c_proj.base_layer.bias', 'decoder.base_model.model.transformer.h.0.attn.c_proj.base_layer.weight', 'decoder.base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight', 'decoder.base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight', 'decoder.base_model.model.transformer.h.0.crossattention.c_attn.base_layer.bias', 'decoder.base_model.model.transformer.h.0.crossattention.c_attn.base_layer.weight', 'decoder.base_model.model.transformer.h.0.crossattention.c_attn.lora_A.

[ERROR] during prediction: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

Generated caption: ['']


../aten/src/ATen/native/cuda/TensorCompare.cu:110: _assert_async_cuda_kernel: block: [0,0,0], thread: [0,0,0] Assertion `probability tensor contains either `inf`, `nan` or element < 0` failed.


In [110]:
# from mlflow.tracking import MlflowClient

# client = MlflowClient()
# versions = [1, 2, 3, 4, 5, 6, 7, 8, 9]
# for version in versions:
#     client.delete_model_version(name="BartPho_ViT_GPT2_LoRA_ICG", version=version)

# client.delete_registered_model(name="BartPho_ViT_GPT2_LoRA_ICG")